In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
from time import sleep

cal = calendar.Calendar()

qlist = ['정치', '경제', '사회', '생활문화', '세계', 'IT과학']
nlist = ['100', '101', '102', '103', '104', '105']
category = 4 
query = qlist[category]
qnum = nlist[category]

def scrape(url):
    import requests
    try:
        res = requests.get('https://news.naver.com'+url, 
                           headers={'User-Agent':'Mozilla/5.0'}, 
                           allow_redirects=False, timeout=10)
        return res.text
    except:
        return "-"
    
if __name__ == '__main__':
    
    for year in range(2005, 2020):
        for month in range(1, 13):
            print(year, month)

            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', 'category', 
                                         '#1_crawling', '#1_'+query, '#1_'+ str(year) + '_'+query, 
                                         '{}.xlsx'.format("#1_"+str(month).zfill(2)+"_"+query))
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.append(["title", "source", "date", "html", "url"])
            wb.save(xlsx_name)

            book = openpyxl.load_workbook(xlsx_name)
            writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
            writer.book = book
            writer.sheets = {ws.title: ws for ws in book.worksheets}

            monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]

            for day in monthdays:

                dd = str(year) + str(month).zfill(2) + str(day).zfill(2)        


                url = "https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId="+qnum+"&date="+dd
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                print(raw)
                html = BeautifulSoup(raw.text, "html.parser")
                articles = html.select("li.ranking_item ")
                title, urls, source, date, records, html, cate = list(), list(), list(), list(), list(), list(), list()

                for ar in articles:

                    title.append(ar.select_one("div.ranking_headline a").get('title'))
                    urls.append(ar.select_one("div.ranking_headline a").get('href'))
                    try:
                        source.append(ar.select_one("div.ranking_office").text)
                    except:
                        source.append(ar.select_one("div.ranking_office"))
                    date.append(dd)
                    cate.append(category)
                
                sleep(5)

                with Pool(10) as p:
                    records = p.map(scrape, urls)
                for re in records:
                    html.append(BeautifulSoup(re, "lxml"))

                d = pd.DataFrame({"title": title, "source": source, 
                                  "date": date, "html": html, "url": urls, 'category': cate}) 
                sleep(0.001)

                for sheetname in writer.sheets:
                        try:
                            d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                        except:
                            pass

            writer.save()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
from time import sleep

cal = calendar.Calendar()

category = 6
query = "연예"

def scrape(url):
    import requests
    try:
        res = requests.get('https://entertain.naver.com'+url, 
                           headers={'User-Agent':'Mozilla/5.0'}, 
                           allow_redirects=False, timeout=10)
        return res.text
    except:
        return "-"
    
if __name__ == '__main__':
    
    for year in range(2014, 2020):
        for month in range(1, 13):
            print(year, month)

            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', 'category', 
                                         '#1_crawling', '#1_'+query, '#1_'+ str(year) + '_'+query, 
                                         '{}.xlsx'.format("#1_"+str(month).zfill(2)+"_"+query))
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.append(["title", "source", "date", "html", "url"])
            wb.save(xlsx_name)

            book = openpyxl.load_workbook(xlsx_name)
            writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
            writer.book = book
            writer.sheets = {ws.title: ws for ws in book.worksheets}

            monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]

            for day in monthdays:

                dd = str(year) + str(month).zfill(2) + str(day).zfill(2)        
            
                url = "https://entertain.naver.com/ranking/page.json?&type=hit_total&date=" + dd
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                print(raw)
                new = raw.json()
                html = BeautifulSoup(new["newsListHtml"], "html.parser")
                articles = html.select('div.tit_area')
                
                title, urls, date, records, html, cate = list(), list(), list(), list(), list(), list()
                
                
                for ar in articles:
                    
                    title.append(ar.select_one("a.tit").text)
                    urls.append(ar.select_one("a").get('href'))
                    date.append(dd)
                    cate.append(category)
                
                sleep(5)
                
                with Pool(10) as p:
                    records = p.map(scrape, urls)
                for re in records:
                    html.append(BeautifulSoup(re, "lxml"))

                d = pd.DataFrame({"title": title, "date": date, "html": html, "url": urls, 'category': cate}) 
                
                sleep(0.001)

                for sheetname in writer.sheets:
                        try:
                            d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                        except:
                            pass

            writer.save()